In [ ]:
# 코드

# 1. 데이터 다운로드 및 압축 해제
!wget -O Aquarium_Data.zip https://app.roboflow.com/ds/54KhujduUB?key=4e4yhwdsYR
import zipfile, os

with zipfile.ZipFile('./Aquarium_Data.zip') as zf:
    zf.extractall('./Aquarium_Data/')

# 2. YAML 생성
!pip install -q PyYAML
import yaml

data = {
    'train': '/content/Aquarium_Data/train/images',
    'val': '/content/Aquarium_Data/valid/images',
    'test': '/content/Aquarium_Data/test/images',
    'names': ['iv_bag', 'saline_level'],
    'nc': 2
}

yaml_path = '/content/Aquarium_Data/Aquarium_Data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

# 3. YOLOv8 설치 및 환경 확인
!pip install -q ultralytics
import ultralytics
ultralytics.checks()

# 4. 모델 불러오기 및 학습
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # 또는 yolov8s.pt (더 큰 모델)
model.train(data=yaml_path, epochs=100, patience=20, batch=8, imgsz=640)

# 5. 검증 데이터셋 평가
metrics = model.val(data=yaml_path)

# 6. 성능 지표 출력
mAP_5 = metrics.results_dict['metrics/mAP50(B)']
mAP_5over = metrics.results_dict['metrics/mAP50-95(B)']
precision = metrics.results_dict['metrics/precision(B)']
recall = metrics.results_dict['metrics/recall(B)']
fitness = metrics.results_dict['fitness']
f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

print(f"mAP@0.5: {mAP_5:.4f}")
print(f"mAP@0.5:0.95: {mAP_5over:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Fitness: {fitness:.4f}")

# 7. 성능 그래프 시각화
import matplotlib.pyplot as plt

metrics_dict = {
    "mAP@0.5": mAP_5,
    "mAP@0.5:0.95": mAP_5over,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1_score,
}

plt.figure(figsize=(8, 5))
plt.bar(metrics_dict.keys(), metrics_dict.values(), color='skyblue')
plt.title("YOLOv8 Evaluation Metrics")
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--')
plt.show()

# 8. 테스트 이미지 예측 및 결과 저장
from glob import glob
test_images = sorted(glob('/content/Aquarium_Data/test/images/*'))
results = model.predict(source='/content/Aquarium_Data/test/images', save=True)

from PIL import Image
from IPython.display import display
import os

image_dir = 'runs/detect/train3'

for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.jpg', '.png')):
        image_path = os.path.join(image_dir, filename)
        print(f"Displaying: {filename}")
        display(Image.open(image_path))

# 10. 예측 클래스별 개수 출력
from collections import Counter

for result in results:
    class_ids = [int(cls) for cls in result.boxes.cls]
    count = Counter(class_ids)
    print(f"\nClass counts: {dict(count)}")
    for idx, class_id in enumerate(class_ids):
        class_name = result.names[class_id]
        conf = float(result.boxes.conf[idx])
        print(f'Class {class_id:>2} ({class_name:<13}) | Confidence: {conf:.2f}')

# 11. 모델 저장
model.save('trained_yolov8n.pt')
